# Overview
This notebook is associated with Trello card https://trello.com/c/y4Nv52JN, specifically the Spain data. 

## Discovery
### Dynamic
The following sources are updated on each run:
1. https://covid19.isciii.es/ provides the following: 
 - autonomous_community_iso
 - date	
 - cases
 - hospitalized	
 - intensive care unit (icu)	
 - deceased	
 - recovered
 
### Static
The following sources were used for one time access:
 1. http://locations.guide-spain.com/ provides the following for the 17 Spanish Kingdoms
  - latitude
  - longitude
 1. http://maps.google.com provides the following for the 2 Spanish Autonomous Cities
  - latitude
  - longitude
 1. https://en.wikipedia.org/wiki/Autonomous_communities_of_Spain provides the following 19 Spanish Autonomous Communities
  - area
  - population
  - density
  - gdp_per_capita_euros

## Read Data

In [120]:
import pandas as pd
from datetime import datetime

In [121]:
url = "https://covid19.isciii.es/resources/serie_historica_acumulados.csv"
df = pd.read_csv(url, encoding="latin_1")
df.shape


(799, 7)

In [122]:
df[df['CCAA Codigo ISO']=='AN'].head(10)

,CCAA Codigo ISO,Fecha,Casos,Hospitalizados,UCI,Fallecidos,Recuperados
0,AN,20/02/2020,NaN,NaN,NaN,NaN,NaN
19,AN,21/02/2020,NaN,NaN,NaN,NaN,NaN
38,AN,22/02/2020,NaN,NaN,NaN,NaN,NaN
57,AN,23/02/2020,NaN,NaN,NaN,NaN,NaN
76,AN,24/02/2020,NaN,NaN,NaN,NaN,NaN
95,AN,25/02/2020,NaN,NaN,NaN,NaN,NaN
114,AN,26/02/2020,1.0,NaN,NaN,NaN,NaN
133,AN,27/02/2020,6.0,NaN,NaN,NaN,NaN
152,AN,28/02/2020,8.0,NaN,NaN,NaN,NaN
171,AN,29/02/2020,12.0,NaN,NaN,NaN,NaN


## Model Data

### Column Headers

In [123]:
df.rename(columns={'CCAA Codigo ISO': 'autonomous_community_iso', 'Fecha': 'date', 'Casos ': 'cases', 'Hospitalizados': 'hospitalized', 'UCI': 'icu', 'Fallecidos': 'deceased', 'Recuperados': 'recovered'}, inplace=True)
print(df.head())

  autonomous_community_iso        date  cases  hospitalized  icu  deceased  \
0                       AN  20/02/2020    NaN           NaN  NaN       NaN   
1                       AR  20/02/2020    NaN           NaN  NaN       NaN   
2                       AS  20/02/2020    NaN           NaN  NaN       NaN   
3                       IB  20/02/2020    1.0           NaN  NaN       NaN   
4                       CN  20/02/2020    1.0           NaN  NaN       NaN   

   recovered  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


### Replace NaN with 0

In [124]:
df['cases'].fillna(value=0, inplace=True)
df['hospitalized'].fillna(value=0, inplace=True)
df['icu'].fillna(value=0, inplace=True)
df['deceased'].fillna(value=0, inplace=True)
df['recovered'].fillna(value=0, inplace=True)

df.head()

,autonomous_community_iso,date,cases,hospitalized,icu,deceased,recovered
0,AN,20/02/2020,0.0,0.0,0.0,0.0,0.0
1,AR,20/02/2020,0.0,0.0,0.0,0.0,0.0
2,AS,20/02/2020,0.0,0.0,0.0,0.0,0.0
3,IB,20/02/2020,1.0,0.0,0.0,0.0,0.0
4,CN,20/02/2020,1.0,0.0,0.0,0.0,0.0


### Update date 
Transform to compliance with https://coronawhy.github.io/task-geo/data_model.html

In [125]:
df['date'] = pd.to_datetime(df['date'])
df.head()

,autonomous_community_iso,date,cases,hospitalized,icu,deceased,recovered
0,AN,2020-02-20,0.0,0.0,0.0,0.0,0.0
1,AR,2020-02-20,0.0,0.0,0.0,0.0,0.0
2,AS,2020-02-20,0.0,0.0,0.0,0.0,0.0
3,IB,2020-02-20,1.0,0.0,0.0,0.0,0.0
4,CN,2020-02-20,1.0,0.0,0.0,0.0,0.0


### Undo cumulative sums

In [126]:
# show current values for AN. See how they are cumulative values
print(df[df['autonomous_community_iso']=='AN'].head(30))

# create a copy of the dataframe, without date
unrolled_df = df.copy()
unrolled_df.drop(['date'], axis=1, inplace=True)

# unroll (i.e. undo the cumulative values)
unrolled_df = unrolled_df.groupby('autonomous_community_iso').diff().fillna(unrolled_df)

# add back autonomous_community_iso, date columns
unrolled_df = pd.concat([df[['autonomous_community_iso', 'date']], unrolled_df], axis=1)

# show the unrolled_df. See how values are no longer cumulative
print(unrolled_df[unrolled_df['autonomous_community_iso']=='AN'].head(30))

    autonomous_community_iso       date   cases  hospitalized   icu  deceased  \
0                         AN 2020-02-20     0.0           0.0   0.0       0.0   
19                        AN 2020-02-21     0.0           0.0   0.0       0.0   
38                        AN 2020-02-22     0.0           0.0   0.0       0.0   
57                        AN 2020-02-23     0.0           0.0   0.0       0.0   
76                        AN 2020-02-24     0.0           0.0   0.0       0.0   
95                        AN 2020-02-25     0.0           0.0   0.0       0.0   
114                       AN 2020-02-26     1.0           0.0   0.0       0.0   
133                       AN 2020-02-27     6.0           0.0   0.0       0.0   
152                       AN 2020-02-28     8.0           0.0   0.0       0.0   
171                       AN 2020-02-29    12.0           0.0   0.0       0.0   
190                       AN 2020-01-03    12.0           0.0   0.0       0.0   
209                       AN

### Insert Country

In [127]:
unrolled_df.insert(0, 'country', 'Spain')

### Remove rows that are not a region. This is significant because the last row includes some text

In [128]:
print('Before: ', unrolled_df.shape)
unrolled_df = unrolled_df[unrolled_df['autonomous_community_iso'] \
  .isin(["CE","AR", "CM", "PV", "MC", "AS", "AN", "CL", "CT", "MD", "IB", "GA", "CN", "VC", "RI", "NC", "EX", "ME", "CB"])]
print('After: ', unrolled_df.shape)

Before:  (799, 8)
After:  (798, 8)


In [129]:
### Add Area, Population, Density, GDP
unrolled_df['area_km_squared'] = unrolled_df['autonomous_community_iso'].map( \
  {\
   "CE": 18.5,
   "AR": 47719,
   "CM": 79463,
   "PV": 7234,
   "MC": 11313,
   "AS": 10604,
   "AN": 87268,
   "CL": 94223,
   "CT": 32114,
   "MD": 8028,
   "IB": 4992,
   "GA": 29574,
   "CN": 7447,
   "VC": 23255,
   "RI": 5045,
   "NC": 10391,
   "EX": 41634,
   "ME": 12.3,
   "CB": 5321
  })
unrolled_df['population'] = unrolled_df['autonomous_community_iso'].map( \
  {\
   "CE": 84777,
   "AR": 1319291,
   "CM": 2032863,
   "PV": 2207776,
   "MC": 1493898,
   "AS": 1022800,
   "AN": 8414240,
   "CL": 2399548,
   "CT": 7675217,
   "MD": 6663394,
   "IB": 1149460,
   "GA": 2699499,
   "CN": 2153389,
   "VC": 5003769,
   "RI": 316798,
   "NC": 654214,
   "EX": 1067710,
   "ME": 86487,
   "CB": 581078
  })
unrolled_df['density_pop_per_km_squared'] = unrolled_df['population']/unrolled_df['area_km_squared']

unrolled_df['gdp_per_capita_euros'] = unrolled_df['autonomous_community_iso'].map( \
  {\
   "CE": 19335,
   "AR": 25540,
   "CM": 17698,
   "PV": 30829,
   "MC": 18520,
   "AS": 21035,
   "AN": 16960,
   "CL": 22289,
   "CT": 27248,
   "MD": 29385,
   "IB": 24393,
   "GA": 20723,
   "CN": 19568,
   "VC": 19964,
   "RI": 25508,
   "NC": 29071,
   "EX": 15394,
   "ME": 16981,
   "CB": 22341
  })
unrolled_df.head()

,country,autonomous_community_iso,date,cases,hospitalized,icu,deceased,recovered,area_km_squared,population,density_pop_per_km_squared,gdp_per_capita_euros
0,Spain,AN,2020-02-20,0.0,0.0,0.0,0.0,0.0,87268.0,8414240,96.418389,16960
1,Spain,AR,2020-02-20,0.0,0.0,0.0,0.0,0.0,47719.0,1319291,27.647080,25540
2,Spain,AS,2020-02-20,0.0,0.0,0.0,0.0,0.0,10604.0,1022800,96.454168,21035
3,Spain,IB,2020-02-20,1.0,0.0,0.0,0.0,0.0,4992.0,1149460,230.260417,24393
4,Spain,CN,2020-02-20,1.0,0.0,0.0,0.0,0.0,7447.0,2153389,289.161944,19568


### Reorder Columns

In [130]:
reordered_df = unrolled_df[['country', 'autonomous_community_iso', 'area_km_squared',\
                           'population', 'gdp_per_capita_euros', 'density_pop_per_km_squared', 'date', 'cases',\
                            'hospitalized', 'icu', 'deceased', 'recovered']]
reordered_df.head()

,country,autonomous_community_iso,area_km_squared,population,gdp_per_capita_euros,density_pop_per_km_squared,date,cases,hospitalized,icu,deceased,recovered
0,Spain,AN,87268.0,8414240,16960,96.418389,2020-02-20,0.0,0.0,0.0,0.0,0.0
1,Spain,AR,47719.0,1319291,25540,27.647080,2020-02-20,0.0,0.0,0.0,0.0,0.0
2,Spain,AS,10604.0,1022800,21035,96.454168,2020-02-20,0.0,0.0,0.0,0.0,0.0
3,Spain,IB,4992.0,1149460,24393,230.260417,2020-02-20,1.0,0.0,0.0,0.0,0.0
4,Spain,CN,7447.0,2153389,19568,289.161944,2020-02-20,1.0,0.0,0.0,0.0,0.0
